In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [5]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [6]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [7]:
print("% of women who survived:", rate_women)
print("% of men who survived:", rate_men)

% of women who survived: 0.7420382165605095
% of men who survived: 0.18890814558058924


In [8]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest,chi2

In [9]:
train_data.isnull().mean()*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [10]:
test_data.isnull().mean()*100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.239234
Cabin          78.229665
Embarked        0.000000
dtype: float64

In [11]:
train_data.drop("Cabin", axis = 1, inplace=True)
test_data.drop("Cabin", axis = 1, inplace=True)

In [12]:
si = SimpleImputer()
oe = OrdinalEncoder(handle_unknown="ignore")
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

In [13]:
train_data["family"] = train_data["SibSp"]+train_data["Parch"]+1
test_data["family"] = test_data["SibSp"]+test_data["Parch"]+1


In [14]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1


In [15]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,2
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,3


In [16]:
train_data.drop(["SibSp","Parch"],axis = 1, inplace=True)
test_data.drop(["SibSp","Parch"],axis = 1, inplace=True)


In [17]:
train_data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,family
348,349,1,3,"Coutts, Master. William Loch ""William""",male,3.0,C.A. 37671,15.9000,S,3
502,503,0,3,"O'Sullivan, Miss. Bridget Mary",female,NaN,330909,7.6292,Q,1
771,772,0,3,"Jensen, Mr. Niels Peder",male,48.0,350047,7.8542,S,1
326,327,0,3,"Nysveen, Mr. Johan Hansen",male,61.0,345364,6.2375,S,1
321,322,0,3,"Danoff, Mr. Yoto",male,27.0,349219,7.8958,S,1


In [18]:
train_data.dropna( subset='Embarked',inplace=True)
test_data.dropna( subset='Embarked',inplace=True)

In [19]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
Ticket           0
Fare             0
Embarked         0
family           0
dtype: int64

In [20]:
train_data.drop(["Name","Ticket","Fare"],axis = 1, inplace=True)
test_data.drop(["Name","Ticket","Fare"],axis = 1, inplace=True)

In [21]:
train_data.sample(5)

,PassengerId,Survived,Pclass,Sex,Age,Embarked,family
396,397,0,3,female,31.0,S,1
408,409,0,3,male,21.0,S,1
682,683,0,3,male,20.0,S,1
864,865,0,2,male,24.0,S,1
380,381,1,1,female,42.0,C,1


In [22]:
test_data.head()

,PassengerId,Pclass,Sex,Age,Embarked,family
0,892,3,male,34.5,Q,1
1,893,3,female,47.0,S,2
2,894,2,male,62.0,Q,1
3,895,3,male,27.0,S,1
4,896,3,female,22.0,S,3


In [23]:
X_train = train_data.drop(["Survived","PassengerId"],axis = 1)
y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId",axis = 1)


In [24]:
trf = ColumnTransformer(transformers=[("age_imputation",si,["Age"]),
                                      ("Sex",ohe,["Sex","Embarked"])],remainder="passthrough")

In [25]:
trf1 = RandomForestClassifier()

In [26]:
trf2 = SelectKBest(score_func=chi2, k=5)

In [27]:
pipe = Pipeline(steps=[("trf",trf),("trf2",trf2)
               ,("trf1",trf1)])

In [28]:
pipe.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_imputation',
                                                  SimpleImputer(), ['Age']),
                                                 ('Sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Sex', 'Embarked'])])),
                ('trf2',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7eda27a59ab0>)),
                ('trf1', RandomForestClassifier())])

In [29]:
prediction = pipe.predict(X_test)

In [30]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
